In [ ]:
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from collections import Counter

from constants import FONT
from constants import COW_RESULTS
from constants import LOCATION_MAPPING_AT
from constants import DF_COW

In [ ]:
df_results = pd.read_csv(COW_RESULTS)

In [ ]:
regions = [[4793, 4274, 4753, 4582, 4794], [3684, 8983, 8962, 6162, 7433]]
plz_map = {4793: 'Farm 1', 4274: 'Farm 2', 4753: 'Farm 3', 4582: 'Farm 4', 4794: 'Farm 5',
           3684: 'Province:<br>Lower Austria', 8983: 'Province:<br>Styria ', 8962: 'Province:<br>Styria', 
           6162: 'Province:<br>Tyrol', 7433: 'Province:<br>Burgenland'}

for idx, region in enumerate(regions):
    fig = go.Figure()

    for duration in df_results['duration'].unique():
        for plz in region:
            if plz == 4794:
                results = df_results.loc[(df_results['duration'] == duration) & (df_results['postal_code'] == plz) & (df_results['animal_id'] == '5bbe020ae89dca5838f92a66')]['result'].to_numpy()
            else:
                results = df_results.loc[(df_results['duration'] == duration) & (df_results['postal_code'] == plz)]['result'].to_numpy()
            x = [[plz_map[plz]]*len(results), ['{:3d} days'.format(int(duration))]*len(results)]
            fig.add_trace(go.Box(y=results, x=x, name='Days: {:3d}'.format(int(duration)), showlegend=False))
            
    fig.update_yaxes(title='L2-Error (km)', range=[0, 125])
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, font=dict(size=16))
    fig.write_image('plots/error_animals_{:}.pdf'.format(idx))
    fig.show()

In [ ]:
mapping = LOCATION_MAPPING_AT
provinces = mapping['province'].unique()

for province in provinces:
    min_plz = mapping.loc[mapping['province'] == province]['zip'].min()
    max_plz = mapping.loc[mapping['province'] == province]['zip'].max()

    results = df_results.loc[(df_results['postal_code'] >= min_plz) &
                             (df_results['postal_code'] <= max_plz) & 
                             (df_results['duration'] == 365)]
    if results.empty:
        continue

    print('{:14}: {:5.2f}, {:5.2f}'.format(province, results['result'].median(), results['result'].mean()))
    
print('-----------------------------')
results = df_results.loc[df_results['duration'] == 365]
print('Total         : {:5.2f}, {:5.2f}'.format(results['result'].median(), results['result'].mean()))
results = df_results.loc[df_results['duration'] == 7]
print('Total (week)  : {:5.2f}, {:5.2f}'.format(results['result'].median(), results['result'].mean()))

In [ ]:
fig = go.Figure()

for duration in df_results['duration'].unique():
    vals = df_results.loc[df_results['duration']==duration]['result'].to_numpy()
    fig.add_trace(go.Box(y=vals, name='Days: {:3d}'.format(int(duration))))

fig.update_yaxes(title='L2-Error (km)')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, font=FONT, template='simple_white')
fig.write_image('plots/error_full.pdf')
#fig.show()

In [ ]:
df_cow = pd.read_csv(DF_COW)

with open('austria.geojson', 'r') as f:
    geojson = json.load(f)

mapping = LOCATION_MAPPING_AT
lang_mapping = {'Vienna': 'Wien', 'Lower Austria': 'Niederösterreich', 'Upper Austria': 'Oberösterreich',
                'Salzburg': 'Salzburg', 'Tyrol': 'Tirol', 'Vorarlberg': 'Vorarlberg', 'Burgenland': 'Burgenland',
                'Styria': 'Steiermark', 'Carinthia': 'Kärnten', 'South Tyrol': 'Südtirol'}

provinces = Counter()
for plz in df_cow['postal_code'].unique():
    province = mapping.loc[mapping['zip'] == plz]['province'].squeeze()
    provinces[province] += 1
provinces['Vorarlberg'] = 0

df = pd.DataFrame([{'Province': p[0], 'Count': p[1]} for p in provinces.items()])
df['Bundesland'] = df['Province'].apply(lambda x: lang_mapping[x])

In [ ]:
box = go.Choroplethmapbox(geojson=geojson, locations=df['Bundesland'], z=df['Count'],
                          featureidkey='properties.name', text=df['Province'], hoverinfo='text')

fig = go.Figure(box)
fig.update_layout(mapbox_style='white-bg', mapbox_zoom=5.8, mapbox_center = {"lat": 47.7, "lon": 13.31})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.write_image('./plots/map_austria.pdf')
#fig.show()